# Parameter types

You can get some of the benefit of Param from Parameter and Parameterized alone, such as having constant or readonly parameters, parameter value inheritance, and parameter docstrings. However, you'll typically want a more specialized Parameter type so that you can enforce type and bounds restrictions and enable suitable behavior specialized for that type of parameter. Param ships with a set of pre-defined more-specific Parameter types, and additional [Custom Parameters](Custom_Parameters.ipynb) can and should be added for any domain-specific parameter types.

The predefined types are organized into a class hierarchy where each type is a subclass of Parameter:

- **String**: String value, optionally constrained by a regular expression
- **Boolean**: True or False (or None, if allowed)
  * **Event**: True/False parameter used to trigger actions (see [Dependencies and watchers])(Dependencies_and_Watchers.ipynb).
- **Color**: A hex string specifying an RGB color, or a standard named color
- **Dynamic**: Base class for values that can be set to a callable that returns a concrete value
  * **Number**: Any type of numeric value
    - **Integer**: Positive or negative integer value
    - **Magnitude**: Positive value in the inclusive range 0.0,1.0
    - **Date**: Date or datetime type
    - **CalendarDate**: Date (not datetime)
- **Callable**: A callable object, such as a function.
  * **Action**: A callable with no arguments, ready to invoke
- **Path**: A POSIX-style string specifying the location of a local file or folder
  * **Filename**: A POSIX-style string specifying the location of a local file
  * **Foldername**: A POSIX-style string specifying the location of a local folder
- **Composite**: A list of other attributes or parameters of this class, settable and readable as a group
- **List**: A list of objects, potentially of a fixed, min, or max length
  * **HookList**: A list of callables, for calling user-defined code at a processing stage
- **Tuple**: Python tuple of a fixed length and optional fixed type
  * **NumericTuple**: Python tuple of a fixed length and a numeric type
    - **XYCoordinates**: Position in an x,y plane
    - **Range**: A numeric range or interval
      * **DateRange**: A range of dates or datetimes
      * **CalendarDateRange**: A range of dates (not datetimes)
- **SelectorBase**: Abstract superclass covering various selector parameter types
  * **Selector**: One object selected out of a provided ordered list of objects
    - **FileSelector**: One filename selected out of those matching a provided glob
    - **ListSelector**: Multiple objects selected out of a provided list of objects
      - **MultiFileSelector**: Multiple filenames selected out of those matching a provided glob
    - [### Actually FileSelector, ListSelector, and MultiFileSelector inherit from ObjectSelector ATM; should be from Selector!]
  * **ClassSelector**: An instance or class of a specified class or superclass
    - **Dict**: A Python dictionary
    - **Array**: NumPy array
    - **DataFrame**: A Pandas DataFrame
    - **DataFrame**: A Pandas Series


The full arguments and behavior of these types is covered in the [Reference Manual](https://param.holoviz.org/Reference_Manual/param.html). Here we will discuss the major categories of Parameter type and how to use them. After reading about Parameters in general, feel free to skip around in this page and only look at the parameter types of interest to you!



## Numbers

A Number is the most common type of Parameter. All numbers in param are of class Dynamic, which allows them to be set not just to a single value but to a value that can repeatedly be drawn from a distribution or a sequence. (See [Dynamic Parameters](Dynamic_Parameters.ipynb) for more information about using these dynamic features, which will not be further discussed here.) Any Number has a default value and optional bounds.

There are two types of bounds: ``bounds`` and ``softbounds``.  ``bounds`` are hard bounds: the parameter must have a value within the specified range.  The default bounds are (None,None), meaning there are actually no hard bounds.  One or both bounds can be set by specifying a value (e.g. bounds=(None,10) means there is no lower bound, and an upper bound of 10). Bounds are inclusive by default, but exclusivity can be specified for each bound by setting inclusive_bounds (e.g. inclusive_bounds=(True,False) specifies an exclusive upper bound). 

When not being dynamically generated, bounds are checked when a Number is created or set. Using a default value outside the hard bounds, or one that is not numeric, results in an exception. When being dynamically generated, bounds are checked when the value of a Number is requested. A generated value that is not numeric, or is outside the hard bounds, results in an exception. 

A separate function set_in_bounds() is provided that will silently crop the given value into the legal range, for use in, for instance, a GUI. 

``softbounds`` are present to indicate the typical range of the parameter, but are not enforced. Setting the soft bounds allows, for instance, a GUI to know what values to display on sliders for the Number. 

``set_hook``???

``step``???

Example of creating a Number::

AB = Number(default=0.5, bounds=(None,10), softbounds=(0,1), doc='Distance from A to B.')


... TO FINISH

## Dates

Date: Date or datetime type
CalendarDate: Date (not datetime)

## Tuples and Lists

`Tuple`, `NumericTuple`, `XYCoordinates`, `Range`, `DateRange`, `CalendarDateRange`

`List`, `HookList`

## Selectors

ClassSelector(SelectorBase):
Dict(ClassSelector):
Array(ClassSelector):
DataFrame(ClassSelector):
ObjectSelector(SelectorBase):
Selector(ObjectSelector):
Series(ClassSelector):
FileSelector(ObjectSelector):
ListSelector(ObjectSelector):
MultiFileSelector(ListSelector):

## Paths

Path(Parameter):
Filename(Path):
Foldername(Path):

## Other

String(Parameter):
Action(Parameter):
Color(Parameter):
Event(Boolean):

... TO FINISH